In [1]:
!pip install datasets
!pip install transformers
!pip install py7zr
!pip install sentencepiece
!pip install rouge-score
!pip install bert-score
!pip install git+https://github.com/google-research/bleurt.git
!pip install nltk
!pip install evaluate

!wget https://storage.googleapis.com/bleurt-oss/bleurt-base-128.zip
!unzip bleurt-base-128.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import numpy as np
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from rouge_score import rouge_scorer
from bert_score import BERTScorer
from bleurt import score as bleurt_score
from nltk.translate.bleu_score import sentence_bleu


In [3]:
import evaluate
bleu = evaluate.load('bleu')
#bleu_score = bleu.compute(references=refs, predictions=preds)

In [12]:
from bleurt import score

# Create a BLEURT scorer
checkpoint = "bleurt-base-128"
bleurt_scorer = score.BleurtScorer(checkpoint)


In [13]:
from datasets import load_metric
rouge = load_metric('rouge')
bleurt_metric = load_metric('bleurt')
from nltk.translate.bleu_score import sentence_bleu
import bert_score
total_scores = {"bleu":0}


def calculate_scores(model, dataloader, tokenizer, device):
    model.eval()

    rouge_scores_list = []
    total_bert_score = 0
    total_bleurt_score = 0
    total_bleu_score = 0
    num_batches = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, target_ids = [b.to(device) for b in batch]

            summary_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
            pred_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
            real_summaries = tokenizer.batch_decode(target_ids, skip_special_tokens=True)

            # ROUGE Scores
            rouge_scores = rouge.compute(predictions=pred_summaries, references=real_summaries)
            rouge_scores_list.append(rouge_scores)

            # BERTScore
            P, R, bert_scores = bert_score.score(pred_summaries, real_summaries, lang="en", model_type='bert-base-uncased', rescale_with_baseline=True)
            total_bert_score += torch.mean(bert_scores).item()

            # Calculate BLEURT
            bleurt_scores = bleurt_scorer.score(references=real_summaries, candidates=pred_summaries)

            mean_bleurt_score = np.mean(bleurt_scores)

            # BLEU Scores
            total_scores["bleu"] += bleu.compute(references=real_summaries, predictions=pred_summaries)['bleu']

            num_batches += 1

    average_bert_score = total_bert_score / num_batches
    average_bleurt_score = total_bleurt_score / num_batches
    total_scores['bleu'] = total_scores['bleu']/num_batches
    # Calculating average ROUGE scores
    rouge1 = np.mean([score['rouge1'].mid.fmeasure for score in rouge_scores_list])
    rouge2 = np.mean([score['rouge2'].mid.fmeasure for score in rouge_scores_list])
    rougeL = np.mean([score['rougeL'].mid.fmeasure for score in rouge_scores_list])

    overall_rouge = np.mean([rouge1, rouge2, rougeL])


    average_scores = {'rouge1': rouge1, 'rouge2': rouge2, 'rougeL': rougeL, 'overall_rouge': overall_rouge, 'bert': average_bert_score, 'bleurt': average_bleurt_score, 'bleu': total_scores['bleu']}

    return average_scores

In [5]:
from datasets import load_metric
rouge = load_metric('rouge')
from sklearn.model_selection import train_test_split
bleurt_metric = load_metric('bleurt')

def fine_tune(model, tokenizer, dataset):
    num_epochs = 5
    batch_size = 3
    learning_rate = 1e-5

    train_dataset = dataset['train']
    validation_dataset = dataset['validation']
    test_dataset = dataset['test']

    tokenized_inputs_train = tokenizer(train_dataset['dialogue'], truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    tokenized_targets_train = tokenizer(train_dataset['summary'], truncation=True, padding='max_length', max_length=64, return_tensors='pt')

    tokenized_inputs_val = tokenizer(validation_dataset['dialogue'], truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    tokenized_targets_val = tokenizer(validation_dataset['summary'], truncation=True, padding='max_length', max_length=64, return_tensors='pt')

    tokenized_inputs_test = tokenizer(test_dataset['dialogue'], truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    tokenized_targets_test = tokenizer(test_dataset['summary'], truncation=True, padding='max_length', max_length=64, return_tensors='pt')

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    train_dataset = torch.utils.data.TensorDataset(tokenized_inputs_train['input_ids'], tokenized_inputs_train['attention_mask'], tokenized_targets_train['input_ids'])
    val_dataset = torch.utils.data.TensorDataset(tokenized_inputs_val['input_ids'], tokenized_inputs_val['attention_mask'], tokenized_targets_val['input_ids'])
    test_dataset = torch.utils.data.TensorDataset(tokenized_inputs_test['input_ids'], tokenized_inputs_test['attention_mask'], tokenized_targets_test['input_ids'])

    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    optimizer = AdamW(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, target_ids = [b.to(device) for b in batch]

            decoder_input_ids = target_ids[:, :-1].contiguous()
            labels = target_ids[:, 1:].clone().detach()
            labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, labels=labels)
            loss = outputs.loss
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        average_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, target_ids = [b.to(device) for b in batch]

                decoder_input_ids = target_ids[:, :-1].contiguous()
                labels = target_ids[:, 1:].clone().detach()
                labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, labels=labels)
                loss = outputs.loss
                total_val_loss += loss.item()

        average_val_loss = total_val_loss / len(val_loader)

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Average Training Loss: {average_train_loss}')
        print(f'Average Validation Loss: {average_val_loss}')

    return model, val_loader,test_loader



In [6]:
# Load the dataset
dataset = load_dataset('samsum')

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [7]:
model_name = 'google/pegasus-large'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Call the function to fine-tune the model
fine_tuned_model, val_loader,test_loader = fine_tune(model, tokenizer, dataset)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5
Average Training Loss: 2.699450346474928
Average Validation Loss: 1.8273317447075477
Epoch 2/5
Average Training Loss: 1.9311009156949115
Average Validation Loss: 1.7127262257831
Epoch 3/5
Average Training Loss: 1.7948532012615348
Average Validation Loss: 1.661748822137113
Epoch 4/5
Average Training Loss: 1.7113835974007607
Average Validation Loss: 1.629933896300557
Epoch 5/5
Average Training Loss: 1.658297312512123
Average Validation Loss: 1.6139273715543223


In [9]:
fine_tuned_model.save_pretrained('fine_tuned_pegasus_large')

In [14]:
# Calculate scores on the test set
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scores = calculate_scores(fine_tuned_model, test_loader, tokenizer,device)




In [15]:
# Print out the scores
print(f'ROUGE-1 Score: {scores["rouge1"]}')
print(f'ROUGE-2 Score: {scores["rouge2"]}')
print(f'ROUGE-L Score: {scores["rougeL"]}')
print(f'Overall ROUGE Score: {scores["overall_rouge"]}')
print(f'BERTScore: {scores["bert"]}')
print(f'BLEURT Score: {scores["bleurt"]}')
print('BLEU Score: {:.4f}'.format(total_scores['bleu']))


ROUGE-1 Score: 0.43339165894055115
ROUGE-2 Score: 0.22440209396763147
ROUGE-L Score: 0.3615211674649861
Overall ROUGE Score: 0.3397716401243896
BERTScore: 0.454774660832716
BLEURT Score: 0.0
BLEU Score: 0.1254


In [16]:
def summarize_text(text, model, tokenizer,device):
    # Input Tokenization
    input_ids = tokenizer.encode(text, truncation=True, max_length=512, return_tensors='pt')

    # Move input_ids to device
    input_ids = input_ids.to(device)


    # Generating summary
    summary_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)

    # Decoding the summary tokens back to text
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    return summary

In [17]:
# Select a sample from the dataset
sample = dataset['test'][0]
input_text = sample['dialogue']

In [18]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary = summarize_text(input_text, fine_tuned_model, tokenizer,device)


In [19]:
  print("Summary:", summary)

Summary: can't find Betty's number. Larry called her last time they were at the park together.
